In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
from src.Particle import ParticleType
from src.JetBuilder import JetBuilder
from src.JetImage import JetImage, JetImageCalculator, JetImageCalculatorPandas, JetImageAvarageCalculator
from src.Preprocessing import PreprocessingJetImages
from sklearn.model_selection import train_test_split


In [2]:
data_gluon = pd.read_csv('Dados/g_jets.csv', header=None, sep=' ')
data_quark = pd.read_csv('Dados/q_jets.csv', header=None, sep=' ')

In [3]:
all_jets = pd.concat([data_gluon, data_quark], axis= 0)
all_jets.reset_index(drop=True, inplace=True)

jets_order = {
    ParticleType.LightQuark: (0, len(data_quark) - 1),
    ParticleType.Gluon: (len(data_quark), len(data_gluon) + len(data_quark) - 1)
}

In [4]:
jet_image_preprocessing = PreprocessingJetImages(phi_range=(-0.4, 0.4),eta_range=(-0.4, 0.4),n_bins_phi=16,n_bins_eta=16)

In [5]:
X =  jet_image_preprocessing.transform(X=all_jets.to_numpy(),y=jets_order)
label = jet_image_preprocessing.jet_labels

In [6]:
x, x_test, y, y_test = train_test_split(X, label, train_size = 0.7, shuffle= True)
vx, tx, vy, ty = train_test_split(x_test, y_test, train_size = 0.5, shuffle= True)

In [9]:
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC 

In [14]:
regularization_rate = 10**(-8)
dropout_rate = 0.3

model = keras.Sequential()
model.add(Input(shape=(16,16)))

model.add(Conv2D(128 , kernel_size=(5, 5), activation='relu',kernel_regularizer=l2(regularization_rate)))
model.add(MaxPool2D(pool_size=(2, 2),strides=(2, 2)))
model.add(Dropout(dropout_rate))

model.add(Conv2D(128 , kernel_size=(5, 5), activation='relu',kernel_regularizer=l2(regularization_rate)))
model.add(MaxPool2D(pool_size=(2, 2),strides=(2, 2)))
model.add(Dropout(dropout_rate))

model.add(Conv2D(64 , kernel_size=(3, 3), activation='relu',kernel_regularizer=l2(regularization_rate)))
model.add(MaxPool2D(pool_size=(2, 2),strides=(2, 2)))
model.add(Dropout(dropout_rate))

model.add(Flatten())

model.add(Dense(128, activation='relu', kernel_regularizer=l2(regularization_rate)))
model.add(Dropout(dropout_rate))

model.add(Dense(2, activation='softmax'))


learning_rate = 0.0001

optimizer = Adam(learning_rate=learning_rate)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=[AUC(name='auc')])

NameError: name 'Input' is not defined

In [11]:
from tensorflow.keras.callbacks import ModelCheckpoint

callbacks = [ModelCheckpoint(
        filepath = "model.keras",
        save_best_only=True,
        monitor = 'val_loss',
                    )]

#foi uma função q achei pra salvar umas infos de maneira mais fácil ( peguei tudo dum tutorial )
history = model.fit(
    x, y,
    epochs = 50,
    validation_data = (vx, vy),
    callbacks = callbacks,
    batch_size = 10 
)

Epoch 1/50


ValueError: Input 0 of layer "conv2d" is incompatible with the layer: expected min_ndim=4, found ndim=2. Full shape received: (None, 256)

In [ ]:
accuracy = history.history['accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
val_accuracy = history.history['val_accuracy']
epochs = range(1,len(accuracy) + 1)
plt.plot(epochs, accuracy,'r',label = 'Train acc')
plt.plot(epochs, val_accuracy,'b',label = 'Validation acc')
plt.xlabel('Epochs')
plt.ylabel('%')
plt.title("Acurácia de treino e validação")
plt.legend()

plt.figure()
plt.plot(epochs,loss,'r',label='Train Loss')
plt.plot(epochs,val_loss,'b',label = 'Validation loss')
plt.xlabel('Epochs')
plt.ylabel('%')
plt.title("Loss de Treino e validação")
plt.legend()
plt.show()


In [12]:
y.shape

(243551, 2)

In [15]:
x.shape

(243551, 256)

In [14]:
from sklearn.metrics import roc_curve

y_pred_prob = model.predict(x_test).ravel()

# Calcular a curva ROC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')

plt.show()